# Iniciando com os submódulos de otimização e rede neural com uma regressão simples
- O título do notebook já diz tudo
- A ideia é abordamos dois submódulos muito importantes:    
    - `torch.nn`: https://pytorch.org/docs/stable/nn.html
        - Contém as funções básicas para construção de redes neurais
    - `torch.optim`: https://pytorch.org/docs/stable/optim.html
        - Contém os algortimos mais utilizados e conhecidos para otimização de redes neurais (ex: SGD)
        
- A ideia não é introduzir todas as funções existentes dentro dos módulos  
- Vamos ir aprendendo aos poucos, conforme vamos introduzindo alguns exemplos e conceitos

In [ ]:
import torch
import torch.nn as nn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Carregando uma base de dados
- Como de costume, vamos carregar uma base do `sklearn` apenas para treinarmos conceitos da Pytorch
    - Obs: Pytorch também tem um submodulo de datasets, mas vamos utilizar em breve

In [ ]:
califa = datasets.fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(califa["data"], califa["target"], test_size=0.25, random_state=8)
X_train.shape, X_test.shape

- Aplicando uma normalização

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

# Definindo uma regressão linear

## Definindo o modelo
- O primeiro passo é definir o modelo da regressão linear
- Dentro do submódulo `torch.nn`, um neurônio perceptron é definido como `torch.nn.Linear()`
    - Basicamente aplica uma transformação linear $y = xA^{\top}+b$
    - [Documentação](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear)

In [ ]:
input_size = 8
output_size = 1
reg_model = nn.Linear(input_size, output_size)
reg_model

### Definindo a função de perda e o algoritmo de otimização
- O segundo passo é definir qual a função de perda vamos utilizar e qual o algoritmo de otimização
- O módulo `torch.nn` fornece um gama de funções de perda
    - [Documentação](https://pytorch.org/docs/stable/nn.html#loss-functions)
    - Para esse exemplo, vamos usar a MSE: `torch.nn.MSELoss()`

In [ ]:
loss_func = nn.MSELoss()

- Agora, precisamos definir um método de otimização
- Como já sabemos, métodos baseados em gradiente são o padrão para otimizarmos algoritmos de machine learning
- Neste exemplo, vamos utilizar o SGD, que já vimos a ideia por trás do algoritmo
    - [Documentação](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)
    - `torch.optim.SGD`
- Para esse método, temos que passar, obrigatoriamente, quais são os parâmetros a serem otimizados e a taxa de aprendizado
    - Os demais parâmetros são opcionais
    
- Para obtermos todos os parametros treinaveis de um modelo (também conhecido como pesos), podemos chamar o método `parameters()` do modelo já declarado

In [ ]:
optimizer = torch.optim.SGD(reg_model.parameters(), lr=0.0001)  

# Loop de treinamento
- Agora que já definimos todos os passos, precisamos fazer nosso loop de treinamento
- Lembre-se que o gradiente descendente é um algoritmo iterativo, precisamos atualizar os pesos a cada época
- Para isso, vamos usar a diferenciação automática

In [ ]:
num_epochs = 10000

for epoch in range(num_epochs):
    
    inputs = torch.from_numpy(X_train_norm).float()
    targets = torch.from_numpy(y_train).float()
    
    # Fazendo a forwardpass
    outputs = reg_model(inputs)
    error = loss_func(outputs.flatten(), targets)
    
    # Agora aplicando a backward pass e fazendo a otimização
    optimizer.zero_grad()
    error.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print (f"Epoch [{epoch+1}/{num_epochs}], MSE: {error.item():.4f}")

## Salvando e carregando um modelo

- Salvando

In [ ]:
torch.save(reg_model, "model.pth")

- Carregando:

In [ ]:
my_model = torch.load("model.pth")
my_model

### Fazendo uma predição

In [ ]:
with torch.no_grad():
    preds = my_model(torch.from_numpy(X_test_norm).float())

- Calculando o erro no conjunto de teste:

In [ ]:
loss_func(preds.flatten(), torch.from_numpy(y_test).float())

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(preds.numpy()[0:100], marker="o", linestyle="dotted", label="Predito", color="b")
ax.plot(y_test[0:100], marker="o", linestyle="dotted", label="Real", color="g")
ax.legend()

___
# Exercícios
1. Aplique a técnica de minibatch no loop de treinamento feito acima
2. Escolha uma base de dados de classificação e aplique uma regressão logística utilizando o que foi aprendido nesse notebook
    - Dicas:
        - Você precisa adicionar uma sigmoid no final da regressão linear
        - Você precisa alterar a função de perda para lidar com problema de classificação
___